# Predicting Product Ratings using Customer Reviews

In [0]:
# start the timer
import time
start_time = time.time()

## Environment Setup

**`Classroom-Setup`**

In [0]:
%run ./Includes/Classroom-Setup

### Loading the DataFrame

In [0]:
textDF = spark.read.csv("/mnt/training/reviews/reviews.csv", inferSchema = True, header=True, escape='"')
display(textDF.limit(10))

Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than most.
2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all","This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar. And it is a tiny mouthful of heaven. Not too chewy, and very flavorful. I highly recommend this yummy treat. If you are familiar with the story of C.S. Lewis' ""The Lion, The Witch, and The Wardrobe"" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch."
4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The flavor is very medicinal.
5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal."
6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,"I got a wild hair for taffy and ordered this five pound bag. The taffy was all very enjoyable with many flavors: watermelon, root beer, melon, peppermint, grape, etc. My only complaint is there was a bit too much red/black licorice-flavored pieces (just not my particular favorites). Between me, my kids, and my husband, this lasted only two weeks! I would recommend this brand of taffy -- it was a delightful treat."
7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,"This saltwater taffy had great flavors and was very soft and chewy. Each candy was individually wrapped well. None of the candies were stuck together, which did happen in the expensive version, Fralinger's. Would highly recommend this candy! I served it at a beach-themed party and everyone loved it!"
8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and chewy. The flavors are amazing. I would definitely recommend you buying it. Very satisfying!!
9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my cats can eat the grass. They love it. I rotate it around with Wheatgrass and Rye too
10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for their digestion. Also good for small puppies. My dog eats her required amount at every feeding.


In [0]:
# check numbers of rows and columns

print((textDF.count(), len(textDF.columns)))

(568454, 10)


In [0]:
# sample 10% data for debugging
textDF = textDF.sample(False, 0.1)

### Selecting Columns

In [0]:
textDF = textDF.select("Id", "ProductId", "Score", "Summary", "Text")

textDF.cache().count()

Out[43]: 57055

In [0]:
textDF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



In [0]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

textDF = textDF.withColumn('Score', col('Score').cast(IntegerType()))

In [0]:
textDF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



## LDA

In [0]:
import sparknlp

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from nltk.corpus import stopwords

from pyspark.ml import Pipeline


In [0]:
## text cleaning annotators & Sentiment Analysis stages
documentAssembler = DocumentAssembler()\
    .setInputCol("Text")\
    .setOutputCol("document")
    
senEmbedder = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

sentimentModel = SentimentDLModel.pretrained(name='sentimentdl_use_twitter', lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

finisher = Finisher() \
    .setInputCols(["sentiment"]) \
    .setOutputCols(["sentiment_result2"]) \
    .setOutputAsArray(False) \
    .setCleanAnnotations(True)

from pyspark.ml.feature import StringIndexer

## string indexer on sentiment results 
sentIndex = StringIndexer(inputCol='sentiment_result2',outputCol='sent_label')

from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer


## LDA pre precessing steps
tokenizer = Tokenizer(inputCol="Text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')


from pyspark.ml.clustering import LDA

# Trains a LDA model.
lda = LDA(k=10, maxIter=10, featuresCol='c_vec')

## assemble features vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
assembler = VectorAssembler(inputCols=['sent_label','topicDistribution'],outputCol='features')

## create pipeline for sentiment analysis, lda, and vector assembler
from pyspark.ml import Pipeline


lda_pipe = Pipeline(stages=[documentAssembler, senEmbedder, sentimentModel, finisher, sentIndex, tokenizer, stopremove, count_vec, lda, assembler])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]


In [0]:
lda_model = lda_pipe.fit(textDF)
processed = lda_model.transform(textDF).persist()

In [0]:
display(processed.limit(10))

Id,ProductId,Score,Summary,Text,sentiment_result2,sent_label,token_text,stop_tokens,c_vec,topicDistribution,features
8,B006K2ZZ7K,5,"Wonderful, tasty taffy",This taffy is so good. It is very soft and chewy. The flavors are amazing. I would definitely recommend you buying it. Very satisfying!!,positive,0.0,"List(this, taffy, is, so, good., , it, is, very, soft, and, chewy., , the, flavors, are, amazing., , i, would, definitely, recommend, you, buying, it., , very, satisfying!!)","List(taffy, good., , soft, chewy., , flavors, amazing., , definitely, recommend, buying, it., , satisfying!!)","Map(vectorType -> sparse, length -> 140052, indices -> List(0, 18, 55, 98, 100, 135, 138, 366, 1189, 3358, 5704, 122795), values -> List(4.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 10, values -> List(0.005566218170119425, 0.005611343533239611, 0.005551693192480247, 0.0056048831503632025, 0.005552638951381288, 0.005562603179898767, 0.9498516681333362, 0.005568093404497909, 0.005562015428270905, 0.005568842856412426))","Map(vectorType -> dense, length -> 11, values -> List(0.0, 0.005566218170119425, 0.005611343533239611, 0.005551693192480247, 0.0056048831503632025, 0.005552638951381288, 0.005562603179898767, 0.9498516681333362, 0.005568093404497909, 0.005562015428270905, 0.005568842856412426))"
13,B0009XLVG0,1,My Cats Are Not Fans of the New Food,"My cats have been happily eating Felidae Platinum for more than two years. I just got a new bag and the shape of the food is different. They tried the new food when I first put it in their bowls and now the bowls sit full and the kitties will not touch the food. I've noticed similar reviews related to formula changes in the past. Unfortunately, I now need to find a new food that my cats will eat.",negative,1.0,"List(my, cats, have, been, happily, eating, felidae, platinum, for, more, than, two, years., i, just, got, a, new, bag, and, the, shape, of, the, food, is, different., they, tried, the, new, food, when, i, first, put, it, in, their, bowls, and, now, the, bowls, sit, full, and, the, kitties, will, not, touch, the, food., i've, noticed, similar, reviews, related, to, formula, changes, in, the, past., unfortunately,, i, now, need, to, find, a, new, food, that, my, cats, will, eat.)","List(cats, happily, eating, felidae, platinum, two, years., got, new, bag, shape, food, different., tried, new, food, first, put, bowls, bowls, sit, full, kitties, touch, food., noticed, similar, reviews, related, formula, changes, past., unfortunately,, need, find, new, food, cats, eat.)","Map(vectorType -> sparse, length -> 140052, indices -> List(14, 23, 24, 33, 42, 50, 54, 83, 89, 107, 136, 168, 188, 202, 277, 429, 432, 549, 568, 787, 886, 1043, 1066, 1611, 1913, 2623, 2636, 2697, 2759, 2922, 4280, 10000, 15228), values -> List(3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 10, values -> List(0.002220026979203955, 0.0022380010864382818, 0.002214219590192951, 0.0022354236094553376, 0.0022146039780849835, 0.0022185836542123736, 0.979998963139558, 0.0022207640924727223, 0.002218339916477742, 0.0022210739539034424))","Map(vectorType -> dense, length -> 11, values -> List(1.0, 0.002220026979203955, 0.0022380010864382818, 0.002214219590192951, 0.0022354236094553376, 0.0022146039780849835, 0.0022185836542123736, 0.979998963139558, 0.0022207640924727223, 0.002218339916477742, 0.0022210739539034424))"
27,B001GVISJM,1,Nasty No flavor,"The candy is just red , No flavor . Just plan and chewy . I would never buy them again",negative,1.0,"List(the, candy, is, just, red, ,, no, flavor, ., just, , plan, and, chewy, ., , i, would, never, buy, them, again)","List(candy, red, ,, flavor, ., , plan, chewy, ., , never, buy)","Map(vectorType -> sparse, length -> 140052, indices -> List(0, 13, 22, 57, 292, 334, 517, 768, 842, 870),

In [0]:
(training,testing) = processed.randomSplit([0.8,0.2], seed=100)

In [0]:
from pyspark.ml.classification import RandomForestClassifier
nb = RandomForestClassifier(numTrees=3, maxDepth=2, featuresCol='features', labelCol="Score", seed=100)
fitted = nb.fit(training)
amazon_prediction = fitted.transform(testing)

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Score", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(amazon_prediction)
print("Accuracy of model at predicting user rating was: {}".format(accuracy))

Accuracy of model at predicting user rating was: 0.6345581802274716
